In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.special import boxcox1p

from heamy.dataset import Dataset
from heamy.estimator import Classifier
from heamy.pipeline import ModelsPipeline

import os

train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

target_values = train.Cover_Type.values

temp = train
train = pd.read_csv('./input/train.csv')

temp.drop(['Cover_Type'],inplace = True,axis = 1)

df =  pd.concat([temp, test], axis=0).reset_index(drop=True)

MemoryError: Unable to allocate 237. MiB for an array with shape (55, 565892) and data type int64

In [ ]:
#Creating new features
df['Radial_Distance_To_Hydrology'] = np.sqrt((df['Horizontal_Distance_To_Hydrology'])**2 + (df['Vertical_Distance_To_Hydrology']**2))
g = df.Elevation.mean()
df['Elevation_normal_sq'] = df['Elevation'].apply(lambda x: (x - g)**2)
df["Slope*Elevation"] = df["Slope"] * df["Elevation"]
df["Vertical_Distance_To_Hydrology"] = abs(df['Vertical_Distance_To_Hydrology'])


In [ ]:
tempdf = df.iloc[:,1:10]
tempdf.head()

In [ ]:
sns.heatmap(tempdf.corr(),annot = True,fmt = '.2f')

In [ ]:
sns.distplot(df.Elevation)

In [ ]:
temp.head()
sns.catplot(x = 'Cover_Type', y = 'Elevation',data = train,kind = 'bar')

In [ ]:
g = train.groupby('Cover_Type').Elevation.agg([np.median])
f = g.index.values
sns.barplot(x = f, y = 'median',data = g)

In [ ]:
g = train.groupby('Cover_Type').Aspect.agg([np.median])
f = g.index.values
sns.barplot(x = f, y = 'median',data = g)

In [ ]:
sns.distplot(df.Slope)

In [ ]:
sns.distplot(df.Horizontal_Distance_To_Hydrology)

In [ ]:
g = train.groupby('Cover_Type').Vertical_Distance_To_Hydrology.agg([np.median])
f = g.index.values
sns.barplot(x = f, y = 'median',data = g)

In [ ]:
df.head()
tempdf = train.iloc[:,7:10]
tempdf['Target'] = target_values
tempdf
tempdf = pd.melt(tempdf,id_vars = ['Target'],var_name = 'a')

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.barplot(x = 'Target', y= 'value',data = tempdf,hue = 'a')

In [ ]:
train.groupby('Cover_Type').sum()

In [ ]:
#Assembling the pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import log_loss

kfold = StratifiedKFold(n_splits = 10)


df.drop(['Id'],inplace = True, axis = 1)

NFOLDS = 5
random_state = 31
x_train = df.iloc[:x_shape]
x_test = df.iloc[x_shape:]
y_train = target_values


In [ ]:
models = [RandomForestClassifier(random_state = random_state),LGBMClassifier(n_estimators = 200, learning_rate = 0.1),
         ExtraTreesClassifier(),GradientBoostingClassifier(random_state = random_state)]
lst = []
for model in models:
    r = cross_val_score(model,x_train,y_train,scoring = 'accuracy',cv = kfold,n_jobs = 4)
    lst.append(r.mean())
results = pd.DataFrame(lst, columns = ['1'])
results

In [ ]:
df = Dataset(x_train,y_train,x_test)

In [ ]:
rf_params = {
    'n_estimators': 200,
    'criterion': 'entropy',
    'random_state': 0
}


et_params = {
    'n_estimators': 200,
    'criterion': 'entropy',
    'random_state': 0
}


lgb_params = {
    'n_estimators': 900, 
    'learning_rate':0.1
}

logr_params = {
        'solver' : 'liblinear',
        'multi_class' : 'ovr',
        'C': 1,
        'random_state': 0}
gbm_params = {
    'n_estimators' : 200,
    'learning_rate' : 0.1
}

In [ ]:
rf = Classifier(dataset = df,estimator = RandomForestClassifier,parameters = rf_params,name = 'rf')
et = Classifier(dataset=df, estimator = ExtraTreesClassifier, parameters=et_params,name='et')   
lgbc = Classifier(dataset=df, estimator=LGBMClassifier, parameters=lgb_params,name='lgbc')
gbm = Classifier(dataset=df,estimator=GradientBoostingClassifier, name='gbm')
logr = Classifier(dataset=df, estimator=LogisticRegression, parameters=logr_params,name='logr')

In [ ]:
pipeline = ModelsPipeline(rf,et,lgbc,gbm,logr)

stack_ds = pipeline.stack(k = NFOLDS, seed = random_state )

In [ ]:
lr = LogisticRegression
lr_params = {'C': 5, 'random_state' : random_state, 'solver' : 'liblinear', 'multi_class' : 'ovr',}
stacker = Classifier(dataset=stack_ds, estimator=lr, parameters=lr_params)

In [ ]:
results = stacker.validate(k=NFOLDS,scorer=log_loss)

In [ ]:
preds_proba = stacker.predict() 
predictions = np.round(np.argmax(preds_proba, axis=1)).astype(int) + 1
predictions